# based-trees

Ideias:
- FEITO: Usar a coluna para no texto
- FEITO: Limpar os dados de conteudo
- FEITO (lucas): ordenar os itens novos que ficaram por ultimo por popularidade
- FEITO: Adicionar o nome da coluna, porque vai indicar o assunto, em conjunto do texto

- tunar os hiperparametros
- usa a imagem alem do texto
- usar algum algoritmo mais simples apenas para usuarios novos, knn para content based
- Entender o parametro exclude_unknowns=True do RatioSplit e se tem alguma forma do DMRL gerar previsoes para itens e usuarios novos 

- paralelizar a previsao
Perguntas:


## Imports

In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import load_data


# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

## Load and process data

### Load data

In [33]:
ratings, content, targets = load_data()

In [34]:
ratings["TimestampDate"] = ratings['Timestamp'].dt.date
ratings.loc[ratings.Rating == 0, "Rating"] = 0.01

C:\Users\sub\AppData\Local\Temp\ipykernel_18892\3531454075.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ratings.loc[ratings.Rating == 0, "Rating"] = 0.01


### Limpar os dados de conteudo

- FEITO - Year: deletar o - no texto '2013–'
- FEITO - Rated: alterar as diversas formas de escrever NA para NA. Esse e um caso especial
- FEITO - alterar as diversas formas de escrever NA para None em todas as coluans
- FEITO - Language: tem varias linhas que possuem bizarices como  'None, English' e 'None, French' , 'English, None'...
- FEITO - Ratings: criar uma coluna para cada chave do dicionario, entender quais sao todas as chaves que existem

-----------------------------------------------
Variaveis que nao precisariam ser tratadas com um bert:
- Metascore
- imdbRating
- Type

In [35]:
content_auxiliar = content.drop(columns=["Poster", "Website", "Response", "Episode", "seriesID", "Season"]).copy()

In [36]:
content_auxiliar['Year'] = content_auxiliar['Year'].str.replace('–', '')

In [37]:
nan = content.totalSeasons.unique()[0]
dict_transform_to_na = {
    "Rated":['N/A', 'Not Rated', 'Unrated', 'UNRATED', 'NOT RATED'],
    "all": [nan, 'N/A', 'None', np.nan],
}

for na_value in dict_transform_to_na["all"]:
    content_auxiliar = content_auxiliar.replace(na_value, None)

for na_value in dict_transform_to_na["Rated"]:
    content_auxiliar['Rated'] = content_auxiliar['Rated'].replace(na_value, None)

In [38]:
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace('None, ', '')
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace(', None', '')

In [39]:
# Entendendo os valores possiveis para a coluna Ratings
# A coluna content_auxiliar.Ratings quarda uma lista que posde ter entre 0 e 3 dicionarios. Cada dicionario possui a chave 'Source', 'Value'.
num_ratings_per_item = []
unique_keys = []
rating_sources = []
rating_values = []

for rating_list in content_auxiliar.Ratings:
    num_ratings_per_item.append(len(rating_list))
    for rating_dict in rating_list:
        for key in rating_dict:
            unique_keys.append(key)
        rating_sources.append(rating_dict['Source'])
        rating_values.append(rating_dict['Value'])

set(rating_sources)

{'Internet Movie Database', 'Metacritic', 'Rotten Tomatoes'}

In [40]:
InternetMovieDatabase_list = []
Metacritic_list = []
RottenTomatoes_list = []
for rating_list in content_auxiliar.Ratings:
    InternetMovieDatabase_list.append(None)
    Metacritic_list.append(None)
    RottenTomatoes_list.append(None)
    for rating_dict in rating_list:
        if rating_dict['Source'] == 'Internet Movie Database':
            InternetMovieDatabase_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Metacritic':
            Metacritic_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Rotten Tomatoes':
            RottenTomatoes_list[-1] = rating_dict['Value']

In [41]:
content_auxiliar['Internet Movie Database'] = InternetMovieDatabase_list
content_auxiliar['Metacritic'] = Metacritic_list
content_auxiliar['Rotten Tomatoes'] = RottenTomatoes_list


In [42]:
RottenTomatoes_list

[None,
 None,
 None,
 None,
 None,
 None,
 '100%',
 '100%',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '96%',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '93%',
 None,
 '89%',
 None,
 None,
 '100%',
 None,
 '96%',
 '100%',
 None,
 None,
 None,
 None,
 '97%',
 None,
 None,
 None,
 '89%',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '95%',
 None,
 None,
 None,
 None,
 '99%',
 None,
 '71%',
 None,
 '92%',
 None,
 '90%',
 None,
 None,
 None,
 None,
 None,
 '95%',
 '100%',
 None,
 None,
 '100%',
 '100%',
 '90%',
 '93%',
 None,
 '86%',
 None,
 None,
 '94%',
 None,
 '87%',
 '91%',
 '100%',
 '97%',
 None,
 '55%',
 None,
 None,
 None,
 '92%',
 '96%',
 None,
 '80%',
 '97%',
 None,
 '83%',
 '100%',
 '92%',
 None,
 None,
 None,
 None,
 '82%',
 None,
 '100%',
 '100%',
 '86%',
 '90%',
 '91%',
 None,
 None,
 '92%',
 '100%',
 '97%',
 None,
 '100%',
 '100%',
 '100%',
 None,
 '100%',
 None,
 '100%',
 '96%',
 '100%',
 None,
 '90%',

In [43]:
content_auxiliar.drop(columns=['Ratings'], inplace=True)

### Apendar a coluna no valor do dataframe

In [44]:
# content_columns = content_auxiliar.columns.to_list()
# content_columns.pop(0)

In [45]:
# for column in content_columns:
#     content_auxiliar[column] = content_auxiliar[column].apply(lambda x: f"{column}: {x}; " if x is not None else f"{column}: unknown value; ")

In [46]:
# content_processed = content_auxiliar[['ItemId']].copy()
# content_processed["text"] = content_auxiliar[content_columns].astype(str).fillna('').agg(' '.join, axis=1)

### Viabiliza as colunas para um modelo tabular

In [47]:
content_auxiliar["Internet Movie Database"] = content_auxiliar["Internet Movie Database"].str.split("/").apply(lambda x: x[0] if x is not None else None)
content_auxiliar["Rotten Tomatoes"].replace('%', "", regex=True, inplace=True)
content_auxiliar["Metacritic"].replace('/100', "", regex=True, inplace=True)
content_auxiliar["imdbVotes"].replace(',', "", regex=True, inplace=True)

C:\Users\sub\AppData\Local\Temp\ipykernel_18892\1294497932.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  content_auxiliar["Rotten Tomatoes"].replace('%', "", regex=True, inplace=True)
C:\Users\sub\AppData\Local\Temp\ipykernel_18892\1294497932.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

In [48]:
content_auxiliar["BoxOffice"].replace(',', "", regex=True, inplace=True)

# O metodo replace nao funcionnou para $, cuidado para nao rodar essa cedula 2 vezes
content_auxiliar["BoxOffice"] = content_auxiliar["BoxOffice"].apply(lambda x: x[1:] if x is not None else None) # troca valores $ por ""


C:\Users\sub\AppData\Local\Temp\ipykernel_18892\817257315.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  content_auxiliar["BoxOffice"].replace(',', "", regex=True, inplace=True)


In [49]:
content_auxiliar["Runtime"].replace(' min', "", regex=True, inplace=True)
mask = content_auxiliar.Runtime.str.contains('h', na=False)
content_auxiliar.loc[mask, "Runtime"] = content_auxiliar.loc[mask, "Runtime"].str.split("h").apply(lambda x: str(int(x[0]) * 60 + int(x[1])))

C:\Users\sub\AppData\Local\Temp\ipykernel_18892\896538434.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  content_auxiliar["Runtime"].replace(' min', "", regex=True, inplace=True)


In [50]:
columns_to_float = ['Runtime', 'Internet Movie Database', 'Metacritic',
                    'Rotten Tomatoes', "imdbVotes", "BoxOffice",
                    "totalSeasons", "imdbRating", "Metascore"]  # Falta tartar Awards
content_auxiliar[columns_to_float] = content_auxiliar[columns_to_float].astype(float)

In [51]:
content_auxiliar.head()

,ItemId,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Metascore,imdbRating,imdbVotes,Type,DVD,BoxOffice,Production,totalSeasons,Internet Movie Database,Metacritic,Rotten Tomatoes
0,c9f0f895fb,Edison Kinetoscopic Record of a Sneeze,1894,None,09 Jan 1894,1.0,"Documentary, Short",William K.L. Dickson,None,Fred Ott,A man (Edison's assistant) takes a pinch of sn...,None,United States,1 win,NaN,5.5,1980.0,movie,None,NaN,None,NaN,5.5,NaN,NaN
1,d3d9446802,Leaving the Factory,1895,None,22 Mar 1895,1.0,"Documentary, Short",Louis Lumière,None,None,A man opens the big gates to the Lumière facto...,None,France,None,NaN,6.9,6633.0,movie,None,NaN,None,NaN,6.9,NaN,NaN
2,c20ad4d76f,The Arrival of a Train,1896,None,25 Jan 1896,1.0,"Documentary, Short","Auguste Lumière, Louis Lumière",None,"Madeleine Koehler, Marcel Koehler, Mrs. August...",A group of people are standing in a straight l...,None,France,None,NaN,7.5,11407.0,movie,None,NaN,None,NaN,7.5,NaN,NaN
3,8e296a067a,The Oxford and Cambridge University Boat Race,1895,None,None,NaN,"Short, News, Sport",Birt Acres,None,None,Although the content of this film is primitive...,None,United Kingdom,None,NaN,4.2,39.0,movie,None,NaN,None,NaN,4.2,NaN,NaN
4,54229abfcf,The House of the Devil,1896,None,24 Dec 1896,3.0,"Short, Horror",Georges Méliès,Georges Méliès,"Jehanne d'Alcy, Jules-Eugène Legris, Georges M...",A bat flies into an ancient castle and transfo...,None,France,None,NaN,6.7,3268.0,movie,None,NaN,None,NaN,6.7,NaN,NaN


In [52]:
variaveis_textos = ["Title", "Plot"]
variaveis_categoricas = ["Rated", "Director", "Production"] # variaveis para tratar "Language", Writer, Actors, Country
variaveis_data = ["Released", "Year", "DVD"] # Todas faltam tratar 

In [53]:
content_auxiliar.drop(columns=columns_to_float+variaveis_textos+variaveis_categoricas).head(2)

,ItemId,Year,Released,Genre,Writer,Actors,Language,Country,Awards,Type,DVD
0,c9f0f895fb,1894,09 Jan 1894,"Documentary, Short",None,Fred Ott,None,United States,1 win,movie,None
1,d3d9446802,1895,22 Mar 1895,"Documentary, Short",None,None,None,France,None,movie,None


In [54]:
content_to_train = content_auxiliar[["ItemId"] + columns_to_float + variaveis_categoricas]

In [55]:
# transforma as colunas em numericas: Runtime

# usar o catboost para categoria e para suportar NA...
# transformar em categorias... Genre, Director, Writer, Actors, Language, Country, Type, Production

# Year transformar em anos ate 2025
# Released, DVD transformar em dias ate 2025

## Train model

### Lightfm

In [56]:
from lightfm.data import Dataset
from lightfm import LightFM

d:\miniconda\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [57]:
content_to_train[columns_to_float] = content_to_train[columns_to_float].fillna(-1)
content_to_train[variaveis_categoricas] = content_to_train[variaveis_categoricas].fillna("unknown")

C:\Users\sub\AppData\Local\Temp\ipykernel_18892\2489804412.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_to_train[columns_to_float] = content_to_train[columns_to_float].fillna(-1)
C:\Users\sub\AppData\Local\Temp\ipykernel_18892\2489804412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_to_train[variaveis_categoricas] = content_to_train[variaveis_categoricas].fillna("unknown")


In [58]:
dataset = Dataset()

# Include all unique item IDs from both ratings and content datasets
all_item_ids = list(set(ratings['ItemId'].unique()).union(content_auxiliar['ItemId'].unique()))
all_item_ids.sort()

In [59]:
content_to_train = content_to_train.sort_values("ItemId", ignore_index=True)

# Verifia se todos os itens estao iguais
content_to_train.ItemId.unique().tolist() == all_item_ids

True

In [60]:
# item_features = set()
# for column in content_to_train.drop(columns=["ItemId"]).columns:
#     item_features.update(content_to_train[column].unique())

In [61]:
item_features = set()
for index, row in content_to_train.drop(columns=["ItemId"]).iterrows():
    # from pdb import set_trace; set_trace()
    item_features.update(row.values)

In [62]:
"unknown" in item_features

True

In [63]:
# Fit the dataset with all user and item IDs
dataset.fit(users=ratings['UserId'].unique(),
            items=list(all_item_ids),
            item_features=item_features) # content_to_train.drop(columns="ItemId"))


In [64]:
features_columns = content_to_train.columns.to_list()
features_columns.pop(0)
item_features_data = []
for index, row in content_to_train.iterrows():
    # from pdb import set_trace; set_trace()
    item_features_data.append((row["ItemId"], row[features_columns].values.tolist()))

In [65]:
# item_features_data = [i for i in zip(content_to_train.ItemId,
#                                      content_to_train.drop(columns="ItemId").values.tolist())]

In [66]:
# item_features_data

In [67]:
item_features_matrix = dataset.build_item_features(item_features_data, normalize=True)

In [68]:
item_features_matrix

<38012x86311 sparse matrix of type '<class 'numpy.float32'>'
	with 329597 stored elements in Compressed Sparse Row format>

In [69]:
(interactions, _) = dataset.build_interactions(
    [(row['UserId'], row['ItemId'], row['Rating']) for _, row in ratings.iterrows()]
)

In [70]:
from lightfm import LightFM
from os import cpu_count

: 

In [ ]:
model = LightFM(loss='warp')

model.fit(interactions, item_features=item_features_matrix, epochs=100, num_threads=cpu_count(), verbose=True)

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

In [141]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [144]:
targets.UserId.map(user_id_map.get)
targets.ItemId.map(item_id_map.get)

0           278
1           555
2          1159
3          1533
4          1755
          ...  
616195    36713
616196    37003
616197    37241
616198    37544
616199    37894
Name: ItemId, Length: 616200, dtype: int64

In [148]:
target_prediction = targets.copy()

In [150]:
target_prediction["Rating"] = model.predict(targets.UserId.map(user_id_map.get).values,
                                            targets.ItemId.map(item_id_map.get).values,
                                            item_features=item_features_matrix,
                                            num_threads=cpu_count())

### Catboost

In [25]:
from catboost import Pool, CatBoostRegressor

In [27]:
train_data = ratings[["UserId", "ItemId", "Rating"]].merge(content_auxiliar[["ItemId"] + columns_to_float + variaveis_categoricas], on="ItemId")
train_data.set_index(["UserId", "ItemId"], inplace=True)

In [30]:
target = "Rating"

In [31]:
model = CatBoostRegressor(
    iterations=1000,
    task_type="GPU",    
    devices='0',
    cat_features=variaveis_categoricas

)

In [ ]:
model.fit(train_data.drop(columns=target),
          train_data[target],
          verbose=True)

### Preve para os dados de teste

In [ ]:
test_data = targets.merge(content_auxiliar[["ItemId"] + columns_to_float + variaveis_categoricas], on="ItemId")
test_data.set_index(["UserId", "ItemId"], inplace=True)

In [44]:
previsao_test = model.predict(test_data)

In [51]:
target_prediction = test_data.reset_index()[["UserId", "ItemId"]]

In [52]:
target_prediction["Rating"] = previsao_test

### Salva a previsao

In [151]:
target_prediction = target_prediction.sort_values(["UserId", "Rating"], ascending=[True, False])

In [152]:
# target_prediction.to_csv("submissoes/catboost_versao_1.csv", index=False)
target_prediction.to_csv("submissoes/lightfm_versao_2.csv", index=False)

In [153]:
target_prediction = target_prediction.drop(columns="Rating")

In [154]:
# target_prediction.to_csv("submissoes/catboost_versao_1_sem_rating.csv", index=False)
target_prediction.to_csv("submissoes/lightfm_versao_2_sem_rating.csv", index=False)